In [1]:
import pandas as pd
from pprint import pprint
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine

class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):        
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                child_str = ' '.join(child_str.split()).strip()
                if child_str:
                    row = (page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str) # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage

fp = open("Goethe-Zertifikat_B1_Wortliste.pdf", 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)
#doc.initialize() # leave empty for no password

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

for page in PDFPage.create_pages(doc):
    interpreter.process_page(page)
    # receive the LTPage object for this page
    device.get_result()

In [2]:
def columntranslation(x):
    if x<129:
        return 1
    if x<300:
        return 2
    if x<410.5:
        return 3
    return 4

def assignblock(x,limits):
    return sum(x>=limits)

def addspace(x):
    output = x.iloc[0]
    for i in range(1,len(x)):
        xi = x.iloc[i]
        if len(output)>=2:
            if (output[-1]=="-") & (output[-2].isalpha()):
                output=output[:-1]+xi
        output+=" "+xi
            
    return output

In [3]:
DF = pd.DataFrame(device.rows)
DF.columns = ["page","xmin","ymin","xmax","ymax","text"]
alphw = DF[(DF["page"]>14)&(DF["page"]<102)&(DF["ymin"]>48.44)&(DF["ymin"]<807)]
alphw["column"]=alphw["xmin"].apply(columntranslation)
alphw["columnaux"]=1-alphw["column"].isin([1,3])
alphw["yabs"]=(850-alphw["ymax"]+850*(alphw["column"]>2.5)+alphw["page"].apply(int)*2000-30000).apply(lambda x: int(round(x,0)))

<ipython-input-3-7631136c7781>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["column"]=alphw["xmin"].apply(columntranslation)
<ipython-input-3-7631136c7781>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["columnaux"]=1-alphw["column"].isin([1,3])
<ipython-input-3-7631136c7781>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [4]:
entries = alphw[alphw["columnaux"]==0]
entries = entries.sort_values(["page","yabs"])
entries["dy"] = entries["yabs"].diff()
entries["groupaux"] = (entries["dy"]>13.05).cumsum()
limits = entries.groupby("groupaux").yabs.min()-4

alphw["block"] = alphw["yabs"].apply(lambda x: assignblock(x,limits))
alphw = alphw.sort_values(["columnaux","yabs","xmin"])#.reset_index(drop=True)
final = alphw.groupby(["columnaux","block"]).agg({"text":addspace}).reset_index(drop=False)

entries  = final[final["columnaux"]==0]
examples = final[final["columnaux"]==1]
ff = pd.merge(entries, examples, on="block")

<ipython-input-4-f07c12b0ca46>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  alphw["block"] = alphw["yabs"].apply(lambda x: assignblock(x,limits))


In [5]:
ff = ff[["text_x","text_y"]]
ff.columns = ["entries","examples"]

In [6]:
b1list = pd.read_pickle("B1translist.pkl")

from google_trans_new import google_translator  
import time
translator = google_translator()

rest_translations = []
for i in ff.loc[2800:,"examples"]:
    time.sleep(3)
    rest_translations.append(translator.translate(i, lang_src='de', lang_tgt='en'))

b1list=pd.concat([b1list,pd.Series(rest_translations)]).reset_index(drop=True)

In [7]:
len(b1list)

2869

In [8]:
b1list.to_pickle("B1translistcomplete.pkl")

In [ ]:
from google_trans_new import google_translator  
  
translator = google_translator()  
translate_text = translator.translate('Hola mundo!', lang_src='es', lang_tgt='en')  
print(translate_text)

In [ ]:
import spacy
nlp = spacy.load("de_core_news_lg")
def extractmainword(x):
    candidates = []
    for i in nlp(x):
        candidates.append(i.lemma_)
    sortedwords = sorted(candidates, key=len)
    return sortedwords[-1]
        

In [ ]:
ff["joiner"]=ff["entries"].apply(extractmainword)

In [ ]:
pd.options.display.max_rows = 400

In [ ]:
len(b1list)

In [ ]:
ff

In [ ]:
from google_trans_new import google_translator  
import time
translator = google_translator()

rest_translations = []
for i in ff.loc[2800:,"examples"]:
    time.sleep(3)
    rest_translations.append(translator.translate(i, lang_src='de', lang_tgt='en'))

In [ ]:
ff["translation"] = pd.concat([b1list,pd.Series(rest_translations)]).reset_index(drop=True)

In [ ]:
ff

In [ ]:
ff["examples_trans"] = ff["examples"].apply(lambda x: translator.translate(x, lang_src='de', lang_tgt='en'))

In [ ]:
ff["examples"].iloc[:4].apply(lambda x: translator.translate(x, lang_src='de', lang_tgt='en'))

In [ ]:
!pip install googletrans

In [ ]:
import translate

In [ ]:
!pip install google_trans_new

In [ ]:
import googletrans
translator = googletrans.Translator()
sentence = translator.translate("Hallo! Mein Name ist Merkel. Wie heisst du?",dest="en",src="ge")
sentence.extra_data

In [ ]:
translate.translator('de', 'en',"Hallo! Mein Name ist Merkel. Wie heisst du?")

In [ ]:
ff

In [ ]:
!pip install deep_translator

In [ ]:
from deep_translator import PonsTranslator
word = 'my boss is hebrew but he is not kind'
translated_word = PonsTranslator(source='english', target='french').translate(word, return_all=False)

In [ ]:
translated_word